# Sesión 5 - Entrenar Word2vec y Doc2Vec desde 0

En este notebook vamos a ver cómo se entrena un modelo sencillo de word2vec eligiendo las dimensiones de los vectores.

Además, se creará un modelo de Doc2Vec a partir de un conjunto de documentos y se verá cómo se puede obtener las similitudes entre documentos.


In [ ]:
# Instalamos gensim si no lo tenemos instalado
!pip3 install -U gensim
# Esto es por si no está ya instalado
!pip3 install -U pandas
!pip3 install -U nltk

# Apartado 1.1 Descargamos un corpus de prueba

Vamos a probar con un corpus de noticias que se encuentra en la URL https://valencia.inf.um.es/valencia-tgine/corpusNoticias.zip

In [ ]:
# Descargamos un corpus de noticias que he creado
!wget --no-check-certificate https://valencia.inf.um.es/valencia-tgine/corpusNoticias.zip
!unzip corpusNoticias.zip > extract.log

Leemos todos lo ficheros y los metemos en una variable *texts*

Tened en cuenta que la codificación de caracteres en estos ficheros es UTF-8. Esto depende del contenido de las web a descargar.

In [3]:
from os import listdir
from os.path import isfile, join

my_path = "corpusNoticias/"
texts = []
for fn in listdir(my_path):
  f = open(my_path+fn, encoding = "utf-8")
  file_content = f.read()
  texts.append(file_content)
  f.close()


# Apartado 1.2 Entrenamos un modelo word2vec a partir del corpus

Aquí vamos a entrenar un modelo word2vec con la librería GENSIM. Como Tokenizer se utilizará el word_tokenize de NLTK, pero se podría usar cualquier otro tokenizador.

In [4]:
from os import listdir
from os.path import isfile, join
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
from nltk.tokenize import word_tokenize


In [ ]:
# Procesamos todos los textos y le aplicamos el word_tokenize de NLTK
nltk.download('punkt_tab')
train_texts=[]
for text in texts:
     train_texts.append(word_tokenize(text.lower()))

In [ ]:
from gensim.models import Word2Vec
# define training data
# train model
# se puede entrenar el modelo con distintos parámetros como el tamaño del vector, tamaño de la ventana, las veces que debe
# aparecer una palabra, etc.
model = Word2Vec(train_texts, vector_size=100, window=10, min_count=1, workers=10)
# summarize the loaded model
print(model)
# save model
model.save('model.bin')


In [ ]:
# Cargamos el modelo guardado
new_model = Word2Vec.load('model.bin')

# Probamos el nuevo modelo
# Imprimimos el vector de la palabra 'energía'
print(model.wv['energía'])


In [ ]:
# Probamos similitudes
new_model.wv.similarity("coronavirus", "covid")

In [ ]:
# Probamos en listar alguna de las palabras más similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'energía'
palabra = 'energía'
print(new_model.wv.most_similar(palabra))

In [ ]:
# Probamos alguna analogía
# Covid es a Vacunas lo que Salud es a ...
print(new_model.wv.most_similar(positive=["salud", "vacunas"], negative=["covid"], topn=10))

# Covid es a Vacunas lo que Guerra es a ...
print(new_model.wv.most_similar(positive=["guerra", "vacunas"], negative=["covid"], topn=10))


In [ ]:
# Probamos a mostrar términos similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'ucrania'
palabra = 'ucrania'
print(new_model.wv.most_similar(palabra))

# Apartado 1.3 Entrenamos un Doc2Vec con los mismos textos


In [12]:
#Import all the dependencies
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

#Necestiamos crear un TaggedDocument para cada uno de los textos indicando un índice de cada texto
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(train_texts)]


In [ ]:
# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 10
vec_size = 100
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm = 1,
                epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    doc2vec_model.train(tagged_data,
                total_examples=doc2vec_model.corpus_count,
                epochs=doc2vec_model.epochs)
    # decrease the learning rate
    doc2vec_model.alpha -= 0.0002
    # fix the learning rate, no decay
    doc2vec_model.min_alpha = model.alpha

doc2vec_model.save("d2v.model")
print("Model Saved")

In [ ]:
from gensim.models import Doc2Vec

doc2vec_model= Doc2Vec.load("d2v.model")
# Probamos a encontrar textos similares a uno dado
query_text = "nadal"
test_data = word_tokenize(query_text)
v1 = doc2vec_model.infer_vector(test_data)

# Encontramos los documentos más similares
similar_doc = doc2vec_model.dv.most_similar(v1)

# Imprimimos los 5 documentos más similares
top5_similar_doc = similar_doc[:5]
print(top5_similar_doc)
for doc in top5_similar_doc:
  print("--------------------------")
  print(texts[int(doc[0])])
  print('Similitud:',doc[1])

